In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

In [3]:
from IPython.display import HTML

In [4]:
from brightness_controller import BrightnessController
from controller import Controller
from mpv import MPVController
from radar import LD2450
from radar_data_collector import RadarDataCollector
from sliding_average import SlidingAverage

In [5]:
#uartdev = "/dev/ttyS0"
uartdev = "/dev/tty.usbserial-14410"

## Sliding average

In [6]:
SlidingAverage.show_check(n=12, ws=4)

 -30 |  -30    0    0    0 |  -30    0    0    0 |  -7.50  -7.50 | yes
  85 |  -30   85    0    0 |  -30   85    0    0 |  13.75  13.75 | yes
-100 |  -30   85 -100    0 |  -30   85 -100    0 | -11.25 -11.25 | yes
  60 |  -30   85 -100   60 |  -30   85 -100   60 |   3.75   3.75 | yes
 -41 |   85 -100   60  -41 |  -41   85 -100   60 |   1.00   1.00 | yes
  -7 | -100   60  -41   -7 |  -41   -7 -100   60 | -22.00 -22.00 | yes
  39 |   60  -41   -7   39 |  -41   -7   39   60 |  12.75  12.75 | yes
 -65 |  -41   -7   39  -65 |  -41   -7   39  -65 | -18.50 -18.50 | yes
  92 |   -7   39  -65   92 |   92   -7   39  -65 |  14.75  14.75 | yes
 -73 |   39  -65   92  -73 |   92  -73   39  -65 |  -1.75  -1.75 | yes
  50 |  -65   92  -73   50 |   92  -73   50  -65 |   1.00   1.00 | yes
 -76 |   92  -73   50  -76 |   92  -73   50  -76 |  -1.75  -1.75 | yes


In [7]:
avg = SlidingAverage(4)
for i in range(10):
    avg.add(i)
    print(f"{i} | {avg.vals} | {avg.avg:.2f}")

0 | [0, 0, 0, 0] | 0.00
1 | [0, 1, 0, 0] | 0.25
2 | [0, 1, 2, 0] | 0.75
3 | [0, 1, 2, 3] | 1.50
4 | [4, 1, 2, 3] | 2.50
5 | [4, 5, 2, 3] | 3.50
6 | [4, 5, 6, 3] | 4.50
7 | [4, 5, 6, 7] | 5.50
8 | [8, 5, 6, 7] | 6.50
9 | [8, 9, 6, 7] | 7.50


## Radar LD2450

In [51]:
r = LD2450(uartdev)

In [52]:
r.uartdev

'/dev/tty.usbserial-14410'

In [157]:
r.verbose

False

In [177]:
print(r.in_waiting)
r.clean()
print(r.in_waiting)

0
0


In [159]:
r.restart()

1.1) Failed to start configuration
No header in response to cmd '00 FF'.


In [178]:
r.get_firmware_version()

'V2.04.23101915'

In [179]:
r.get_mac_address()

'F8 38 08 8E BB CB'

In [180]:
r.get_bluetooth_state()

True

In [181]:
r.get_tracking_mode()

2

In [182]:
r.get_zone_filtering()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [183]:
r.show_info()

UART device: /dev/tty.usbserial-14410
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: F8 38 08 8E BB CB)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [184]:
r.set_bluetooth_on(restart=True)
r.set_multi_tracking()

In [185]:
r.show_info()

1.1) Failed to end configuration
No header in response to cmd '00 FE'.
1.2) Failed to end configuration
No header in response to cmd '00 FE'.
1.3) Failed to end configuration
No header in response to cmd '00 FE'.
1.4) Failed to end configuration
No header in response to cmd '00 FE'.
1) Failed to end configuration while executing cmd '00 C1'.
UART device: /dev/tty.usbserial-14410
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: F8 38 08 8E BB CB)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [186]:
r.test(100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
1.1) Failed to start configuration
No acknowledge in response to cmd '00 FF'.
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99

Firmware version: ['V2.04.23101915']
MAC address: ['F8 38 08 8E BB CB']
Tracking mode: [2]


In [219]:
print(r.in_waiting)

939


In [225]:
print(r.in_waiting)
r.show_data(50)

930
 1124  1307 |     0     0 |     0     0 | IN:   990 | sample:     1
 1129  1308 |     0     0 |     0     0 | IN:   960 | sample:     2
 1137  1307 |     0     0 |     0     0 | IN:   930 | sample:     3
 1147  1305 |     0     0 |     0     0 | IN:   900 | sample:     4
 1152  1303 |     0     0 |     0     0 | IN:  1003 | sample:     5
 1155  1301 |     0     0 |     0     0 | IN:   973 | sample:     6
 1158  1300 |     0     0 |     0     0 | IN:   943 | sample:     7
 1162  1298 |     0     0 |     0     0 | IN:   913 | sample:     8
 1168  1298 |     0     0 |     0     0 | IN:  1016 | sample:     9
 1175  1298 |     0     0 |     0     0 | IN:   986 | sample:    10
 1181  1299 |     0     0 |     0     0 | IN:   956 | sample:    11
 1184  1299 |     0     0 |     0     0 | IN:   926 | sample:    12
 1178  1305 |     0     0 |     0     0 | IN:   896 | sample:    13
 1170  1312 |     0     0 |     0     0 | IN:   999 | sample:    14
 1153  1318 |     0     0 |     0     0 | IN

## Radar 2410C

In [226]:
# ...

## Radar 2461

In [227]:
# ...

## Radar data collector

In [228]:
rdc = RadarDataCollector(uartdev)

Radar UP.


In [229]:
rdc.radar.show_info()

1.1) Failed to start configuration
No header in response to cmd '00 FF'.
UART device: /dev/tty.usbserial-14410
Firmware version: V2.04.23101915
Bluetooth: ON (MAC address: F8 38 08 8E BB CB)
Multi tracking: ON
Zone filtering: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [230]:
rdc.uartdev

'/dev/tty.usbserial-14410'

In [231]:
rdc.default_bluetooth

True

In [232]:
rdc.default_multi_tracking

True

In [233]:
rdc.active

True

In [234]:
rdc.qsize

0

In [235]:
rdc.empty

True

In [236]:
rdc.start()

In [237]:
for _ in range(10):
    print(rdc.qsize)
    time.sleep(1)

113
124
135
147
158
169
180
192
203
214


In [238]:
rdc.active = False

Radar DOWN.


In [239]:
for _ in range(10):
    print(rdc.qsize)
    time.sleep(1)

238
238
238
238
238
238
238
238
238
238


## Camera

In [240]:
# ...

## Direct brightness

In [241]:
i2c_num = 2
br_ctl = BrightnessController(i2c_num)

In [242]:
for i in range(0, 101):
    br_ctl.set(i)
    time.sleep(0.01)

FileNotFoundError: [Errno 2] No such file or directory: '/dev/i2c-2'

## Manipulating gamma

In [243]:
# ...

## MPV

In [6]:
mpv_ctl = MPVController()

In [7]:
# mpv_ctl.get_property("working-directory")

In [8]:
# mpv_ctl.get_property("term-status-msg")

In [9]:
# mpv_ctl.socket.recv(1024)

In [10]:
# problems = []
# for i in range(500):
#     res = mpv_ctl.load_file(video_fpath)
#     if res is not None and res['error'] == 'success':
#         continue

#     problems.append(i)

# print(problems)

In [142]:
mpv_ctl.set_property("loop-file", "inf")

{'request_id': 902, 'error': 'success'}

In [21]:
mpv_ctl.clear()

{'data': None, 'request_id': 1, 'error': 'success'}

In [11]:
mpv_ctl.load_file(video_fpath)

{'data': {'playlist_entry_id': 1}, 'request_id': 1, 'error': 'success'}

In [14]:
mpv_ctl.get_property("duration")

{'data': 888.48, 'request_id': 4, 'error': 'success'}

In [15]:
mpv_ctl.get_property("playlist/count")

{'data': 1, 'request_id': 5, 'error': 'success'}

In [16]:
mpv_ctl.get_property("playlist/0/filename")

{'data': '/home/tami/videos/head_on_floor_1920x1080.mp4',
 'request_id': 6,
 'error': 'success'}

In [17]:
mpv_ctl.play()

{'request_id': 7, 'error': 'success'}

In [18]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 8, 'error': 'success'}

In [12]:
mpv_ctl.pause()

{'request_id': 2, 'error': 'success'}

In [140]:
mpv_ctl.get_property("pause")

{'data': False, 'request_id': 900, 'error': 'success'}

In [139]:
mpv_ctl.get_property("time-pos")

{'data': 4.541583, 'request_id': 899, 'error': 'success'}

In [75]:
mpv_ctl.get_property("playback-time")

{'data': -0.004625, 'request_id': 634, 'error': 'success'}

In [138]:
mpv_ctl.get_property("time-remaining")

{'data': 9.893542, 'request_id': 898, 'error': 'success'}

In [137]:
mpv_ctl.get_property("framedrop")

{'data': 'vo', 'request_id': 897, 'error': 'success'}

In [136]:
mpv_ctl.get_property("frame-drop-count")

{'data': 4, 'request_id': 896, 'error': 'success'}

In [113]:
mpv_ctl.seek(0)

{'data': None, 'request_id': 873, 'error': 'success'}

In [116]:
mpv_ctl.get_property("playback-time")

{'data': 7.87825, 'request_id': 876, 'error': 'success'}

In [110]:
mpv_ctl.get_property("time-remaining")

{'data': 4.012667, 'request_id': 870, 'error': 'success'}

In [145]:
for i in range(-100, 1):
    mpv_ctl.set_brightness(i, osd=True)
    time.sleep(0.1)

In [153]:
mpv_ctl.get_property("volume")

{'data': 100.0, 'request_id': 115, 'error': 'success'}

## Controller

In [ ]:
ws = 1
dmin = 1000
dmax = 2500

ctl = Controller(uartdev, ws, dmin, dmax, screen_bus=i2c_num)

In [ ]:
ctl.start()

## Plotter

In [ ]:
p = Plotter(uartdev, ws, dmin, dmax, verbose=True, xmin=-3000, xmax=3000, ymin=-4000)

In [ ]:
ani = p.start()

In [ ]:
HTML(ani.to_jshtml())